# DICOMs to JPGs using FastAI method

### JPGs are generated for both scan images and scans segmented using Watershed-Segmentation with markers

The process followed for generating the JPGs is from this amazing kernel by [Jeremy Howard](https://www.kaggle.com/jhoward) - https://www.kaggle.com/jhoward/don-t-see-like-a-radiologist-fastai. I've also explained breifly why this process is required.

For generating the segemented lung images Marker controlled watershed Segmentation is used from the kernel https://www.kaggle.com/aadhavvignesh/lung-segmentation-by-marker-controlled-watershed by [Aadhav Vignesh](https://www.kaggle.com/aadhavvignesh)

## Datasets Used

**DICOM Metadata** - generated using FastAI
https://www.kaggle.com/gautham11/osic-pulmonary-fibrosis-progression-dicom-metadata

**FastAI2 whls** - https://www.kaggle.com/vijayabhaskar96/fastai2-wheels

## Dataset Generated

**JPG images dataset** - https://www.kaggle.com/gautham11/pulmonaryfibrosisprogressionsegmentationjpgs

### Key Insight

While normalizing the floating point image array to `np.uint8` for storing as jpg, care must be taken to not scale every image by it's `mean` and `std.dev`, because by this the mapping from DICOM values to normalized values will change from image to image. 

In [ ]:
import os
import cv2

import pydicom
import pandas as pd
import numpy as np 
import torch
import tensorflow as tf 
import matplotlib.pyplot as plt 
from pathlib import Path
import scipy.ndimage as ndimage
from skimage import measure, morphology, segmentation
from scipy.ndimage.interpolation import zoom
from PIL import Image 

from tqdm.notebook import tqdm
%matplotlib inline

In [ ]:
!pip install ../input/fastai2-wheels/fastcore-0.1.18-py3-none-any.whl -q
!pip install ../input/fastai2-wheels/fastai2-0.0.17-py3-none-any.whl -q

In [ ]:
from fastai2.basics import *
from fastai2.medical.imaging import *
import cv2
import torch

In [ ]:
data_dir = Path('/kaggle/input/osic-pulmonary-fibrosis-progression/')
patient_paths = (data_dir/'train').ls()
sample_patient = patient_paths[0]

In [ ]:
## Generated by
# dicom_meta = pd.DataFrame()
# for patient in tqdm(patient_paths):
#     dicom_meta = pd.concat([dicom_meta, pd.DataFrame.from_dicoms(patient.ls())], ignore_index=True)

dicom_meta = pd.read_pickle('../input/osic-pulmonary-fibrosis-progression-dicom-metadata/train_dicom_df')

In [ ]:
dicom_meta.head().T

In [ ]:
## Code from https://www.kaggle.com/aadhavvignesh/lung-segmentation-by-marker-controlled-watershed 

def generate_markers(image):
    """
    Generates markers for a given image.
    
    Parameters: image
    
    Returns: Internal Marker, External Marker, Watershed Marker
    """
    
    #Creation of the internal Marker
    marker_internal = image < -400
    marker_internal = segmentation.clear_border(marker_internal)
    marker_internal_labels = measure.label(marker_internal)
    
    areas = [r.area for r in measure.regionprops(marker_internal_labels)]
    areas.sort()
    
    if len(areas) > 2:
        for region in measure.regionprops(marker_internal_labels):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       marker_internal_labels[coordinates[0], coordinates[1]] = 0
    
    marker_internal = marker_internal_labels > 0
    
    # Creation of the External Marker
    external_a = ndimage.binary_dilation(marker_internal, iterations=10)
    external_b = ndimage.binary_dilation(marker_internal, iterations=55)
    marker_external = external_b ^ external_a
    
    # Creation of the Watershed Marker
    marker_watershed = np.zeros(image.shape, dtype=np.int)
    marker_watershed += marker_internal * 255
    marker_watershed += marker_external * 128
    
    return marker_internal, marker_external, marker_watershed


def seperate_lungs(image, iterations = 1):
    """
    Segments lungs using various techniques.
    
    Parameters: image (Scan image), iterations (more iterations, more accurate mask)
    
    Returns: 
        - Segmented Lung
        - Lung Filter
        - Outline Lung
        - Watershed Lung
        - Sobel Gradient
    """
    
    # Store the start time
    # start = time.time()
    
    marker_internal, marker_external, marker_watershed = generate_markers(image)
    
    
    '''
    Creation of Sobel Gradient
    '''
    
    # Sobel-Gradient
    sobel_filtered_dx = ndimage.sobel(image, 1)
    sobel_filtered_dy = ndimage.sobel(image, 0)
    sobel_gradient = np.hypot(sobel_filtered_dx, sobel_filtered_dy)
    sobel_gradient *= 255.0 / np.max(sobel_gradient)
    
    
    '''
    Using the watershed algorithm
    
    
    We pass the image convoluted by sobel operator and the watershed marker
    to morphology.watershed and get a matrix matrix labeled using the 
    watershed segmentation algorithm.
    '''
    watershed = morphology.watershed(sobel_gradient, marker_watershed)
    
    '''
    Reducing the image to outlines after Watershed algorithm
    '''
    outline = ndimage.morphological_gradient(watershed, size=(3,3))
    outline = outline.astype(bool)
    
    
    '''
    Black Top-hat Morphology:
    
    The black top hat of an image is defined as its morphological closing
    minus the original image. This operation returns the dark spots of the
    image that are smaller than the structuring element. Note that dark 
    spots in the original image are bright spots after the black top hat.
    '''
    
    # Structuring element used for the filter
    blackhat_struct = [[0, 0, 1, 1, 1, 0, 0],
                       [0, 1, 1, 1, 1, 1, 0],
                       [1, 1, 1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1, 1, 1],
                       [0, 1, 1, 1, 1, 1, 0],
                       [0, 0, 1, 1, 1, 0, 0]]
    
    blackhat_struct = ndimage.iterate_structure(blackhat_struct, iterations)
    
    # Perform Black Top-hat filter
    outline += ndimage.black_tophat(outline, structure=blackhat_struct)
    
    '''
    Generate lung filter using internal marker and outline.
    '''
    lungfilter = np.bitwise_or(marker_internal, outline)
    lungfilter = ndimage.morphology.binary_closing(lungfilter, structure=np.ones((5,5)), iterations=3)
    
    '''
    Segment lung using lungfilter and the image.
    '''
    segmented = np.where(lungfilter == 1, image, -2000*np.ones(image.shape))
    
    return segmented, lungfilter, outline, watershed, sobel_gradient


In [ ]:
#DICOM Read utils
def fix_pxrepr(dcm):
    if dcm.PixelRepresentation != 0 or dcm.RescaleIntercept<-100:
        return dcm
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000
    return dcm

def read_dcm(path):
    dcm = fix_pxrepr(Path(path).dcmread())
    if dcm.Rows != 512 or dcm.Columns != 512: 
        dcm.zoom_to((512,512))
    return dcm


# Generate JPGs
def get_maskedlung_scan_img(path):
    dcm = read_dcm(path)
    _, lungmask, _, _, _ = seperate_lungs(dcm.scaled_px.numpy())
    masked_lung = torch.tensor(np.where(lungmask, dcm.scaled_px, -2048))
    return dcm, masked_lung

def save_maskedlung_lungimg(path, masked_path, img_path, masked_bins=None, bins=None):
    dcm, masked_lung = get_maskedlung_scan_img(path)
    save_jpg(masked_lung, masked_path, masked_bins)
    save_jpg(dcm, img_path, bins)
    
    
def save_jpg(img, path, bins):
    windows = [dicom_windows.lungs,dicom_windows.subdural]
    img.save_jpg(path, windows, bins=bins)
    
def read_jpg(path):
    return np.array(Image.open(str(path)))

## Generating Bins for Histogram scaling

To convert an image from Housfield values to a [0, 1] range, we split the Housfield pixel values of images into bins, such that each bin has an equal number of pixels. In order to maintain the same mapping from Hounsfield values to normalized value for all images, these bin boundaries are generated from Hounsfield values of multiple sampled images. This process is explained in detail in  https://www.kaggle.com/jhoward/don-t-see-like-a-radiologist-fastai

In [ ]:
dicom_meta['image_format'] = dicom_meta.PixelRepresentation.astype(str) + '_' + dicom_meta.BitsStored.astype(str)
dicom_meta['image_format'].value_counts().to_dict()

In [ ]:
format_df = dicom_meta[['image_format', 'fname']]
selected_files = []
for img_format, n in format_df['image_format'].value_counts().to_dict().items():
    selected_files += list(format_df.loc[format_df['image_format'] == str(img_format)].sample(min(1000, n // 3))['fname'])
    
selected_files[0:5]

In [ ]:
def get_scan_freqhist_bins():
    sample_images = []
    for path in tqdm(selected_files):
        try:
            dcm = read_dcm(path)
            sample_images.append(dcm.scaled_px)
        except Exception as e:
            print(e)
    samples_arr = torch.stack(sample_images)
    del sample_images
    bins = samples_arr.freqhist_bins()
    del samples_arr
    return bins

In [ ]:
# bins = get_scan_freqhist_bins()

# generated from the above code
bins = torch.tensor([-4096., -3024., -2048., -2000., -1109., -1025., -1024., -1023., -1020.,
        -1017., -1014., -1011., -1008., -1005., -1003., -1001.,  -998.,  -996.,
         -993.,  -991.,  -988.,  -985.,  -981.,  -978.,  -973.,  -969.,  -964.,
         -958.,  -952.,  -946.,  -940.,  -933.,  -926.,  -919.,  -912.,  -904.,
         -895.,  -886.,  -875.,  -862.,  -847.,  -829.,  -806.,  -777.,  -738.,
         -684.,  -607.,  -501.,  -386.,  -301.,  -247.,  -210.,  -183.,  -163.,
         -148.,  -136.,  -126.,  -117.,  -109.,  -101.,   -93.,   -85.,   -77.,
          -69.,   -60.,   -51.,   -42.,   -33.,   -24.,   -15.,    -6.,     0.,
            5.,    13.,    20.,    27.,    34.,    41.,    48.,    55.,    62.,
           70.,    79.,    90.,   102.,   118.,   137.,   162.,   193.,   237.,
          301.,   417.,   750.,  1278.])

plt.plot(bins, torch.linspace(0,1,len(bins)));


In [ ]:
def get_maskedscan_freqhist_bins():
    sample_masked_images = []
    for path in tqdm(selected_files):
        try:
            dcm = read_dcm(path)
            dcm_np = dcm.scaled_px.numpy()
            _, lungmask, _, _, _ = seperate_lungs(dcm_np)
            masked_lung = torch.tensor(np.where(lungmask, dcm_np, -2048))
            sample_masked_images.append(masked_lung)
        except Exception as e:
            print(e)
    sample_mask_arr = torch.stack(sample_masked_images)
    del sample_masked_images
    masked_bins = sample_mask_arr.freqhist_bins()
    del sample_mask_arr
    return masked_bins

In [ ]:
# masked_bins = get_maskedscan_freqhist_bins()

masked_bins = torch.tensor([-2048., -1000.,  -942.,  -908.,  -879.,  -847.,  -808.,  -751.,  -656.,
         -467.,   -81.,   251.])

plt.plot(masked_bins, torch.linspace(0,1,len(masked_bins)));


### Testing on sample Patient

In [ ]:
dcom = read_dcm(data_dir/'train/ID00228637202259965313869/100.dcm')

In [ ]:
sample_patient

In [ ]:
fig, axes = plt.subplots(2,2, figsize=[10, 10])
axes[0][0].imshow(dcom.scaled_px)
axes[0][0].set_title('Housfield values')
axes[0][1].imshow(dcom.scaled_px.hist_scaled(bins))
axes[0][1].set_title('Hist Scaled')
axes[1][0].hist(dcom.scaled_px.flatten(), bins=bins)
axes[1][1].hist(dcom.scaled_px.hist_scaled(bins).flatten())

In [ ]:
_, lungmask, _, _, _ = seperate_lungs(dcom.scaled_px.numpy())
masked_lung = torch.tensor(np.where(lungmask, dcom.scaled_px, -2048))

In [ ]:
fig, axes = plt.subplots(2,2, figsize=[10, 10])
axes[0][0].imshow(masked_lung)
axes[0][0].set_title('Housfield values')
axes[0][1].imshow(masked_lung.hist_scaled(masked_bins))
axes[0][1].set_title('Hist Scaled')
axes[1][0].hist(masked_lung.flatten(), bins=masked_bins)
axes[1][1].hist(masked_lung.hist_scaled(bins).flatten())

## Normalized Array to JPGs

In order to fully utilize the 3 channels of JPG image, the first 2 channels are filled with windowed CT scans - https://radiopaedia.org/articles/windowing-ct, and the last channel is filled with histogram scaled values with the generated `bins` 

In [ ]:
nchan_img = dcom.to_nchan([dicom_windows.lungs,dicom_windows.subdural], bins)
nchan_masked_img = masked_lung.to_nchan([dicom_windows.lungs,dicom_windows.subdural], masked_bins)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(nchan_img[0,:,:])
axes[0].set_title('dicom_windows.lungs')
axes[1].imshow(nchan_img[1,:,:])
axes[1].set_title('dicom_windows.subdural')
axes[2].imshow(nchan_img[2,:,:])
axes[2].set_title('hist_scaled')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(nchan_masked_img[0,:,:])
axes[0].set_title('dicom_windows.lungs')
axes[1].imshow(nchan_masked_img[1,:,:])
axes[1].set_title('dicom_windows.subdural')
axes[2].imshow(nchan_masked_img[2,:,:])
axes[2].set_title('hist_scaled')

## Generate Dataset

In [ ]:
!mkdir 'osic-pulmonary-fibrosis-progression-segmentation-jpgs'
!mkdir 'osic-pulmonary-fibrosis-progression-segmentation-jpgs/train'
!mkdir 'osic-pulmonary-fibrosis-progression-segmentation-jpgs/train/scans'
!mkdir 'osic-pulmonary-fibrosis-progression-segmentation-jpgs/train/masked_scans'

In [ ]:
output_dir = Path('osic-pulmonary-fibrosis-progression-segmentation-jpgs/train')
output_dir_scans = output_dir/'scans'
output_dir_mscans = output_dir/'masked_scans'

In [ ]:
patient_dirs = (data_dir/'train/').ls()
len(patient_dirs)

In [ ]:
import traceback
import shutil

def create_a_zip_archive(i, patient_dir_batch):
    for patient in tqdm(patient_dir_batch):
        patient_id = str(patient).split('/')[-1]
        (output_dir_scans/patient_id).mkdir(parents=True, exist_ok=True)
        (output_dir_mscans/patient_id).mkdir(parents=True, exist_ok=True)
        for img_path in patient.glob('*'):
            try:
                img_id = str(img_path).split('/')[-1][:-4]
                if not (output_dir_mscans/patient_id/f'{img_id}.jpg').exists():
                    save_maskedlung_lungimg(
                        img_path, 
                        output_dir_mscans/patient_id/f'{img_id}.jpg',
                        output_dir_scans/patient_id/f'{img_id}.jpg',
                        masked_bins,
                        bins
                    )
            except Exception as e:
                traceback.print_exc()
                print(e)
                print(img_path)
    shutil.make_archive(f'lung_segmentation_scans_{i}', 'zip', str(output_dir))
    shutil.rmtree(str(output_dir_scans))
    shutil.rmtree(str(output_dir_mscans))

In [ ]:
n = 10
start_n = 0
for i, n_start in enumerate(range(start_n, len(patient_dirs), n)):
    create_a_zip_archive(i + int(start_n / n), patient_dirs[n_start: n_start + n])
    # Link to generated dataset added
    break